In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import transforms
BATCH_SIZE = 128
EPOCHS = 10
LEARNING_RATE = 0.001

In [2]:
def train_one_epoch(model, data_loader, loss_fn, optimiser, device):
  for inputs, targets in data_loader:
    inputs,targets = inputs.to(device), targets.to(device)

    #calulate loss
    pred = model(inputs)
    loss = loss_fn(pred, targets)

    #backpropagate loss and update weights
    optimiser.zero_grad()
    loss.backward()
    optimiser.step()
  print(f"Loss: {loss.item()}")

def train(model, data_loader, loss_fn, optimiser, device, epochs):
  for i in range (epochs):
    print(f"Epoch {i+1}")
    train_one_epoch(model, data_loader, loss_fn, optimiser, device)
    print("-------------------------------")
  print("Training is done")


In [3]:
class FeedForwardNet(nn.Module):
  def __init__(self):
    super().__init__()
    self.flatten = nn.Flatten()
    self.dense_layers = nn.Sequential (
        nn.Linear(28*28, 256),
        nn.ReLU(),
        nn.Linear(256,10)
    )
    self.softmax = nn.Softmax(dim=1)
  def forward(self, input_data):
    f_data = self.flatten(input_data)
    logits = self.dense_layers(f_data)
    pred = self.softmax(logits)
    return pred

In [5]:
if __name__ == "__main__":
  train_loader = DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.ToTensor()),
    batch_size=BATCH_SIZE, shuffle=True)
  
  if torch.cuda.is_available():
    device = "cuda"
  else:
    device = "cpu"
  print(f"Using {device} device")

  feed_forward_net = FeedForwardNet().to(device)
  #instantiate loss function + optimiser
  loss_fn = nn.CrossEntropyLoss()
  optimiser = torch.optim.Adam(feed_forward_net.parameters(),
                               lr = LEARNING_RATE)

  #train model
  train(feed_forward_net, train_loader, loss_fn, optimiser, device, EPOCHS)
  
  torch.save(feed_forward_net.state_dict(), "feedforwardnet.pth")
  print("Model trained and stored at feedforwardnet.pth")

Using cuda device
Epoch 1
Loss: 1.5333150625228882
-------------------------------
Epoch 2
Loss: 1.5227593183517456
-------------------------------
Epoch 3
Loss: 1.514814019203186
-------------------------------
Epoch 4
Loss: 1.4947515726089478
-------------------------------
Epoch 5
Loss: 1.511922836303711
-------------------------------
Epoch 6
Loss: 1.4890552759170532
-------------------------------
Epoch 7
Loss: 1.4920458793640137
-------------------------------
Epoch 8
Loss: 1.4797621965408325
-------------------------------
Epoch 9
Loss: 1.4886113405227661
-------------------------------
Epoch 10
Loss: 1.4867969751358032
-------------------------------
Training is done
Model trained and stored at feedforwardnet.pth
